In [ ]:
import polars as pl

# Paths

In [ ]:
# imports csv
base = "../game-dump/dumps/"
dump_imports_p = base + "dump-imports.csv"
old_iat_p = base + "old-iat.csv"
byte_calls_p = base + "broken-byte-calls.csv"
inst_calls_p = base + "broken-analyzed-calls.csv"

In [ ]:
names_map_p = "../game-dump/names-map.csv"
names_map = pl.read_csv(names_map_p)

# Parsing import table

In [ ]:
dump_imports = pl.read_csv(dump_imports_p).filter(pl.col("Module") != "game.exe")

In [ ]:
dump_imports = dump_imports.with_columns(pl.col("Function").str.replace_all(" ", ""))
names_map = names_map.with_columns(
    pl.col("undecorated").str.replace_all(" ", "").alias("Function")
)

dump_imports = dump_imports.join(names_map.drop("undecorated"), on="Function", how="left").with_columns(
    pl.coalesce("decorated", "Function").alias("Function")
).drop("decorated")

In [ ]:
dump_imports = dump_imports.with_columns(("0x" + pl.col("Address").str.to_lowercase()).alias("Address"))
dump_imports = dump_imports.unique("Address", keep='first')

In [ ]:
dump_imports.head()

# Gathering imports from old IAT

In [ ]:
iat = pl.read_csv(old_iat_p)

iat = iat.rename({"Address": "Calladdr", "Destination": "Address"})
iat = iat.with_columns(("0x" + pl.col("Address").str.to_lowercase()).alias("Address"))

iat = iat.with_columns((pl.col("Address").shift(+1).str.slice(2).str.to_integer(base=16) == 0).alias("first").fill_null(True))
iat = iat.with_columns((pl.col("Address").shift(-1).str.slice(2).str.to_integer(base=16) == 0).alias("last").fill_null(True))

iat = iat.join(dump_imports, on='Address', how='left')

In [ ]:
iat.drop("first", "last").write_csv(old_iat_p + '2.csv')

In [ ]:
pl.Config(tbl_rows=40)

In [ ]:
iat2 = iat.filter(pl.col("Function").is_null())
iat2 = iat2.filter(pl.col("Address").str.slice(2).str.to_integer(base=16) != 0)

<0x023e3673> points to an intermediate call which i believe, is an obfuscated jump to user32.dll!wsprintfA

In [ ]:
iat.head()